### Embeddings

In [2]:
pip install typing-extensions --upgrade

  Using cached typing_extensions-4.8.0-py3-none-any.whl (31 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.7.4.3
    Uninstalling typing-extensions-3.7.4.3:
      Successfully uninstalled typing-extensions-3.7.4.3
Note: you may need to restart the kernel to use updated packages.


In [1]:
!pip install -U sentence-transformers

     |████████████████████████████████| 85 kB 2.8 MB/s eta 0:00:01
     |████████████████████████████████| 7.9 MB 4.3 MB/s eta 0:00:011     |█████████████████████████▉      | 6.4 MB 10.0 MB/s eta 0:00:01
     |████████████████████████████████| 146.7 MB 131 kB/s eta 0:00:01
     |████████████████████████████████| 1.7 MB 104 kB/s eta 0:00:01
     |████████████████████████████████| 1.2 MB 9.0 MB/s eta 0:00:01
     |████████████████████████████████| 301 kB 652 kB/s eta 0:00:01
     |████████████████████████████████| 166 kB 2.1 MB/s eta 0:00:01
     |████████████████████████████████| 2.5 MB 218 kB/s eta 0:00:011
     |████████████████████████████████| 439 kB 2.0 MB/s eta 0:00:01
     |████████████████████████████████| 295 kB 11.1 MB/s eta 0:00:01
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125918 sha256=cba72f2a1071be9966e02e99fc070f85fde7d8a4b2de495cf0f06958d5fa1bac
  Stored in directory: /Users/linatobon/Library/Caches/pip/wheels/5

In [3]:
from sentence_transformers import SentenceTransformer,util

model = SentenceTransformer("all-MiniLM-L6-v2")

sentences = ['el gato juega afuera',
             'me gusta tocar guitarra',
             'me encanta la pasta',
             'esta pelicula estuvo asombrosa',
             'el perro esta afuera',
             'te gusta la musica?',
             'estuvo muy buena la pelicula',
             'te gustaria una pizza?']
embeddings = model.encode(sentences)

In [6]:
len(embeddings)

8